### Import Modules, Data Sources and Functions

In [1]:
import requests
import numpy as np
import pandas as pd
import folium
import geopandas as gpd
import json
import time
from yelpapi import YelpAPI
import pickle
import branca

Read geometric files and IRS Data

In [2]:
census_zips_bounds = gpd.read_file('/Volumes/data/cb_2017_us_zcta510_500k/cb_2017_us_zcta510_500k.shp')
census_zips_bounds = census_zips_bounds.set_index('ZCTA5CE10')
census_zips_bounds = census_zips_bounds[['GEOID10','geometry']]
census_zips_bounds.columns = ['regionid','geometry']
census_zips_bounds['regionid'] = census_zips_bounds['regionid'].astype(int)
census_zips_bounds.crs = {'init' :'epsg:4326'}
IRS_data = pd.read_csv('../Data/IRS_with_target.csv')
IRS_data.set_index('zip',inplace=True)

Function for pulling listings from the yelp api

In [3]:
def yelp_s(location):
    response = []
    start_time = time.time()
    location = location
    for i in range(20):
        offsets = i*50
        print('Getting Yelp Businesses Page ' +str(offsets)+' from ' +location+'.')
        iteration = yelp_api.search_query(location=location,
                                     price=[1,2,3,4],
                                     offset=offsets,
                                     limit=50)
        
        response.extend(iteration['businesses'])
    print(f'Total Time Spent: {round((time.time()-start_time))} seconds')
    return response

Function for plotting individual yelp listings, according to their latitude/longitude and price.

In [4]:
def point_plotter(specific_frame,the_map):
    
    
    specific_frame1 = specific_frame.loc[specific_frame['price'] == 1]
    specific_frame2 = specific_frame.loc[specific_frame['price'] == 2]
    specific_frame3 = specific_frame.loc[specific_frame['price'] == 3]
    specific_frame4 = specific_frame.loc[specific_frame['price'] == 4]
    
    name1 = list(specific_frame1["name"])
    lat1 = list(specific_frame1["latitude"])
    lon1 = list(specific_frame1["longitude"])

    name2 = list(specific_frame2["name"])
    lat2 = list(specific_frame2["latitude"])
    lon2 = list(specific_frame2["longitude"])

    name3 = list(specific_frame3["name"])
    lat3 = list(specific_frame3["latitude"])
    lon3 = list(specific_frame3["longitude"])

    name4 = list(specific_frame4["name"])
    lat4 = list(specific_frame4["latitude"])
    lon4 = list(specific_frame4["longitude"])
    
    fg1 = folium.FeatureGroup(name="1 Dollar Sign")
    fg2 = folium.FeatureGroup(name="2 Dollar Signs")
    fg3 = folium.FeatureGroup(name="3 Dollar Signs")
    fg4 = folium.FeatureGroup(name="4 Dollar Signs")
    
    for lt, ln, nm in zip(lat1, lon1, name1):
        fg1.add_child(folium.CircleMarker(location=[lt, ln], radius = 5, popup= nm + ", " + str(1)+" price",
        fill_color='red', fill=True,  color = 'red', fill_opacity=0.7))

    for lt, ln, nm in zip(lat2, lon2, name2):
        fg2.add_child(folium.CircleMarker(location=[lt, ln], radius = 5, popup= nm + ", " + str(2)+" price",
        fill_color='orange', fill=True,  color = 'orange', fill_opacity=0.7))

    for lt, ln, nm in zip(lat3, lon3, name3):
        fg3.add_child(folium.CircleMarker(location=[lt, ln], radius = 5, popup= nm + ", " + str(3)+" price",
        fill_color='yellow', fill=True,  color = 'yellow', fill_opacity=0.7))

    for lt, ln, nm in zip(lat4, lon4, name4):
        fg4.add_child(folium.CircleMarker(location=[lt, ln], radius = 5, popup= nm + ", " + str(4)+" price",
        fill_color='green', fill=True,  color = 'green', fill_opacity=0.7))
        
    the_map.add_child(fg1)
    the_map.add_child(fg2)
    the_map.add_child(fg3)
    the_map.add_child(fg4)

Function for plotting information pulled from yelp

In [5]:
def yelp_plotter(zipcode,a_map):

    df = pd.DataFrame(yelp_s(zipcode))
    df = df.replace({'price' : { "$$$$" : 4, '$$$' : 3, '$$' : 2 , '$': 1}})
    df['latitude'] = df['coordinates'].apply(lambda x: float(x['latitude']))
    df['longitude'] = df['coordinates'].apply(lambda x: float(x['longitude']))
    df['zip_code'] = df['location'].apply(lambda x: x['zip_code'])
    
    df = df.loc[df['zip_code'] == zipcode]
    
    boundaries = census_zips_bounds.loc[census_zips_bounds['regionid'] == int(zipcode)]
    
    bounds = folium.GeoJson(
    boundaries,
    style_function=lambda feature: {
        'fillColor': 'red',
        'color': 'black',
        'weight': 2,
        'dashArray': '5, 5',
        'fillOpacity': 0.3,
        },
        name='asdf'
    )
    
    target = IRS_data['target'][int(zipcode)]
    
    
    price_listing = df.groupby('price')['price'].count()
    
    if 1 in price_listing.keys():
        one_dollar = price_listing[1]
    else:
        one_dollar = 0
    
    if 2 in price_listing.keys():
        two_dollar = price_listing[2]
    else:
        two_dollar_dollar = 0
    
    if 3 in price_listing.keys():
        three_dollar = price_listing[3]
    else:
        three_dollar = 0
        
    if 4 in price_listing.keys():
        four_dollar = price_listing[4]
    else:
        four_dollar = 0
    
    html = f"""
    <h3>Zipcode: {zipcode}</h3>
    <h4>Income Class: {target}</h4>
    <h4>One Dollar: {one_dollar}</h4>
    <h4>Two Dollar: {two_dollar}</h4>
    <h4>Three Dollar: {three_dollar}</h4>
    <h4>Four Dollar: {four_dollar}</h4>
    """
    boundaries = boundaries.reset_index()
    
    bounds.add_to(a_map)
    
    point_plotter(df,a_map)
    
    folium.Marker([boundaries['geometry'][0].centroid.y,boundaries['geometry'][0].centroid.x], popup=html
    ).add_to(a_map)
    
    a_map.add_child(folium.LayerControl())
    
    a_map = folium.Map([boundaries['geometry'][0].centroid.y,boundaries['geometry'][0].centroid.x], zoom_start=12)

# Choose Zipcode and Create the Map

In [6]:
zipcode = '91765'

In [7]:
zipcode_lat = census_zips_bounds['geometry'][zipcode].centroid.y
zipcode_long = census_zips_bounds['geometry'][zipcode].centroid.x

In [8]:
map_2 = folium.Map([zipcode_lat, zipcode_long], zoom_start=12)

In [9]:
personal_key = 'IeU8hcjxedL3QqVVFPsFIvpQYWW2w525k9Pqf99XDbtBCVC4VXU7pmk5XiGHQxyVvLKr08PnXVlfzXsf7Va-7EnQ-_kxQzrRHW7B0lWX_dtAyrm1jw8GPC436LUuXHYx'

In [10]:
yelp_api = YelpAPI(personal_key,timeout_s=3.0)
results = []

In [11]:
yelp_plotter(zipcode,map_2)

Getting Yelp Businesses Page 0 from 91765.
Getting Yelp Businesses Page 50 from 91765.
Getting Yelp Businesses Page 100 from 91765.
Getting Yelp Businesses Page 150 from 91765.
Getting Yelp Businesses Page 200 from 91765.
Getting Yelp Businesses Page 250 from 91765.
Getting Yelp Businesses Page 300 from 91765.
Getting Yelp Businesses Page 350 from 91765.
Getting Yelp Businesses Page 400 from 91765.
Getting Yelp Businesses Page 450 from 91765.
Getting Yelp Businesses Page 500 from 91765.
Getting Yelp Businesses Page 550 from 91765.
Getting Yelp Businesses Page 600 from 91765.
Getting Yelp Businesses Page 650 from 91765.
Getting Yelp Businesses Page 700 from 91765.
Getting Yelp Businesses Page 750 from 91765.
Getting Yelp Businesses Page 800 from 91765.
Getting Yelp Businesses Page 850 from 91765.
Getting Yelp Businesses Page 900 from 91765.
Getting Yelp Businesses Page 950 from 91765.
Total Time Spent: 22 seconds


In [12]:
map_2

Future Goals:
    - Scale to accomodate a list of zipcodes
    - Pop ups with optional Vincent plots